In [ ]:
import kfp
from kfp import components
from networkx import DiGraph
from jinja2 import Environment

In [ ]:
downloader_component_path = "_components/downloader.yaml"
download_document = components.load_component_from_file(downloader_component_path)

transformer_component_path = "_components/transformer.yaml"
transform_document = components.load_component_from_file(transformer_component_path)

scriptwriter_component_path  = "_components/scriptwriter.yaml"
scriptwriter = components.load_component_from_file(scriptwriter_component_path)

performer_component_path = "_components/performer.yaml"
performer = components.load_component_from_file(performer_component_path)

In [ ]:
class Step:
    def __init__(self, component: components.YamlComponent):
        self.component = component

    @property
    def name(self) -> str:
        return self.component.name

    @property
    def description(self) -> str:
        return self.component.description

    @property
    def inputs(self) -> list[str]:
        return list(self.component.component_spec.inputs.keys())
    
    @property
    def outputs(self) -> list[str]:
        return list(self.component.component_spec.outputs.keys())
    
    @property
    def produced_artifacts(self) -> dict[str, kfp.dsl.structures.OutputSpec]:
        return {f"{artifact}": self.component.component_spec.outputs[artifact] for artifact in self.component.component_spec.outputs
                if self.component.component_spec.outputs[artifact].type.startswith("system.")}
    
    @property
    def pipeline_parameters(self) -> dict[str, kfp.dsl.structures.InputSpec]:
        return {f"{self.name}-{param}": self.component.component_spec.inputs[param] for param in self.component.component_spec.inputs
                if not self.component.component_spec.inputs[param].type.startswith("system.")}

    def __repr__(self) -> str:
        return f"Step(name={self.name}, description={self.description})"

In [ ]:
download_document_step = Step(download_document)
transform_document_step = Step(transform_document)
scriptwriter_step = Step(scriptwriter)
performer_step = Step(performer)

In [ ]:
pipe = DiGraph()

pipe.add_node(download_document_step)
pipe.add_node(transform_document_step)
pipe.add_node(scriptwriter_step)
pipe.add_node(performer_step)
pipe.add_edge(download_document_step, transform_document_step)
pipe.add_edge(transform_document_step, scriptwriter_step)
pipe.add_edge(scriptwriter_step, performer_step)

In [ ]:
with open("pipeline.jinja", "r") as f:
    template_str = f.read()

env = Environment(trim_blocks=True, lstrip_blocks=True)
    
# Create the template from our template string
template = env.from_string(template_str)

In [ ]:
# Render the template with components
rendered_yaml = template.render(pipe=pipe)

# Format the rendered YAML
with open("my_pipe.yaml", 'w') as f:
    f.write(rendered_yaml)